## Etapa 1: Importação das bibliotecas

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from torch import nn, optim
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
torch.__version__

'1.13.1+cu116'

## Etapa 2: Base de dados

In [ ]:
torch.manual_seed(123)
np.random.seed(124)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base = pd.read_csv('/content/drive/MyDrive/Deep Learing de A à Z com PyTorch/Bases/personagens.csv')
previsores = base.iloc[:, 0:6].values
classe = base.iloc[:, 6].values

In [ ]:
encoder = LabelEncoder() 
classe = encoder.fit_transform(classe)

In [ ]:
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.25)

In [ ]:
previsores_treinamento = torch.tensor(np.array(previsores_treinamento), dtype = torch.float)
classe_treinamento = torch.tensor(np.array(classe_treinamento), dtype = torch.float)

In [ ]:
dataset = torch.utils.data.TensorDataset(previsores_treinamento, classe_treinamento)
train_loader = torch.utils.data.DataLoader(dataset, batch_size = 10, shuffle = True)

## Etapa 3: Construção do modelo

In [ ]:
classificador = nn.Sequential(
        nn.Linear(6, 4),
        nn.ReLU(),
        nn.Linear(4, 4),
        nn.ReLU(),
        nn.Linear(4, 1),
        nn.Sigmoid()
        )

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(classificador.parameters())

## Etapa 4: Treinamento do modelo

In [ ]:
for epoch in range(2000):
    
    running_loss = 0.
    running_accuracy = 0.
    
    for data in train_loader:
        inputs, labels = data

        optimizer.zero_grad()        

        outputs = classificador(inputs)
        loss = criterion(outputs, labels.float().view(*outputs.shape))
        loss.backward()
        
        optimizer.step()

        outputs = outputs.flatten().detach().numpy() # Representar outputs como um vetor unidimensional
        outputs = (outputs > 0.5).astype('int32') # Converter outputs > 0.5 para classe = 1
        labels = labels.flatten().detach().numpy() # labels também vira vetor unidimensional
        accuracy = accuracy_score(labels, outputs)

        running_loss += loss.item()
        running_accuracy += accuracy

    print('\rÉPOCA {:3d} - perda {:03.2f} - precisão {:03.2f}'.format(epoch + 1, running_loss/len(train_loader), running_accuracy/len(train_loader)), end = '\r')

## Etapa 5: Avaliação do modelo

In [ ]:
classificador.eval()

Sequential(
  (0): Linear(in_features=6, out_features=4, bias=True)
  (1): ReLU()
  (2): Linear(in_features=4, out_features=4, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4, out_features=1, bias=True)
  (5): Sigmoid()
)

In [ ]:
previsores_teste = torch.tensor(np.array(previsores_teste), dtype = torch.float)
classe_teste = torch.tensor(np.array(classe_teste), dtype = torch.float)

In [20]:
outputs = classificador(previsores_teste)
loss = criterion(outputs, classe_teste.float().view(*outputs.shape)).item()
labels = classe_teste.flatten().detach().numpy()
outputs = outputs.flatten().detach().numpy()
outputs = (outputs > 0.5).astype('int32')
accuracy = accuracy_score(labels, outputs)

print('Perda no dataset de teste: ', loss, '\nPrecisão no dataset de teste: ', accuracy)

Perda no dataset de teste:  0.22193484008312225 
Precisão no dataset de teste:  0.918918918918919
